<a href="https://colab.research.google.com/github/TiagoIesbick/dashboard-etl/blob/main/previsoes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import pathlib

In [ ]:
%%capture
!pip install tabula-py[jpype]
import re
import pandas as pd
from tabula import read_pdf
from datetime import datetime
import pathlib
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression

In [ ]:
#retirando colunas nulas
def excluir_colunas_nulas(df: pd.DataFrame) -> pd.DataFrame:
    for coluna in df:
        if df[coluna].notnull().sum() == 0:
            df = df.drop(coluna, axis=1)
    return df

#preenchendo com 0 as células vazias após a primeira célula preenchida em uma coluna
def preencher_0(df: pd.DataFrame) -> pd.DataFrame:
    for coluna in df.iloc[:, 2:]:
        for i, linha in enumerate(df[coluna]):
            if df.loc[i, coluna] > 0:
                df.loc[i:, coluna] = df.loc[i:, coluna].fillna(0)
                break
    return df

def str_to_float(value: str) -> float:
  value = str(value).replace('.', '').replace(',', '.')
  value = re.sub(r'[A-Za-z]|\s', '', value)
  return float(0) if value == '' else float(value)

def agrupamento_format(value: str) -> str:
  if re.search(r'^0{2}', value[:2]):
    value = '   ' + value
  elif re.search(r'^[1-9]\d', value[:2]):
    value = '      ' + value
  return value

dict_month = {'Janeiro': '01', 'Fevereiro': '02', 'Março': '03', 'Abril': '04',
              'Maio': '05', 'Junho': '06', 'Julho': '07', 'Agosto': '08',
              'Setembro': '09', 'Outubro': '10', 'Novembro': '11', 'Dezembro': '12'}

In [155]:
df_exp = pd.read_csv(r'/content/drive/MyDrive/Dashboard_data/final_data/df_exp.csv', sep=';', parse_dates=['Comp.pagto.'])
df_exp.drop('Rubrica', axis=1, inplace=True)

In [156]:
df_exp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51881 entries, 0 to 51880
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Comp.pagto.   51881 non-null  datetime64[ns]
 1   Result. pago  51881 non-null  float64       
 2   Unid. Orçam.  51881 non-null  int64         
 3   Proj/Ativ     51881 non-null  int64         
 4   Vinc. Orçam.  51881 non-null  int64         
 5   Elemento      51881 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(4)
memory usage: 2.4 MB


In [161]:
df_7001 = df_exp.loc[
    (df_exp['Unid. Orçam.'] == 7001) &
    (df_exp['Vinc. Orçam.'].isin([400, 1, 6069])) &
    (df_exp['Comp.pagto.'].dt.year > 2011) &
    (~df_exp['Proj/Ativ'].isin([9042]))
].copy()

df_7001['Comp.pagto.'] = df_7001['Comp.pagto.'].dt.to_period('M')
df_7001.loc[df_7001['Proj/Ativ'] == 2870, 'Proj/Ativ'] = 4396
df_7001.loc[df_7001['Proj/Ativ'].isin([2872, 1507]), 'Proj/Ativ'] = 4471
df_7001.loc[df_7001['Proj/Ativ'].isin([2873, 2532]), 'Proj/Ativ'] = 4413
df_7001.loc[df_7001['Proj/Ativ'].isin([1505, 1503, 1373, 1506]), 'Proj/Ativ'] = 2529
df_7001.loc[(df_7001['Proj/Ativ'] == 2681) & (df_7001['Elemento'] == 319192), 'Elemento'] = 319113
df_7001.loc[(df_7001['Proj/Ativ'] == 2529) & (df_7001['Elemento'] == 449092), 'Elemento'] = 449051
df_7001.loc[(df_7001['Proj/Ativ'] == 9071) & (df_7001['Elemento'] == 319091), 'Elemento'] = 339091
df_7001.loc[(df_7001['Proj/Ativ'] == 9071) & (df_7001['Elemento'].isin([339092, 339147])), 'Elemento'] = 339047
df_7001.loc[~((df_7001['Proj/Ativ'] == 9071) & (df_7001['Vinc. Orçam.'] == 1)), 'Vinc. Orçam.'] = 6069

In [146]:
df_7001.loc[:, 'col'] = df_7001['Unid. Orçam.'].astype(str) + '-' + df_7001['Proj/Ativ'].astype(str) + '-' + df_7001['Elemento'].astype(str) + '-' + df_7001['Vinc. Orçam.'].astype(str)
df_7001['col']

,col
17710,7001-2529-339039-6069
17712,7001-2529-339039-6069
17713,7001-2529-319013-6069
17714,7001-2529-449052-6069
17715,7001-2529-449052-6069
...,...
51866,7001-2529-339033-6069
51867,7001-4471-339040-6069
51868,7001-2680-319013-6069
51878,7001-2529-339039-6069


In [163]:
df_7001

,Comp.pagto.,Result. pago,Unid. Orçam.,Proj/Ativ,Vinc. Orçam.,Elemento
17710,2012-01,270.00,7001,2529,6069,339039
17712,2012-01,7.60,7001,2529,6069,339039
17713,2012-01,829.84,7001,2529,6069,319013
17714,2012-01,9084.00,7001,2529,6069,449052
17715,2012-01,2370.00,7001,2529,6069,449052
...,...,...,...,...,...,...
51866,2025-04,1681.00,7001,2529,6069,339033
51867,2025-04,553.33,7001,4471,6069,339040
51868,2025-04,1314.58,7001,2680,6069,319013
51878,2025-04,1000.00,7001,2529,6069,339039


In [128]:
df_7001.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11108 entries, 17710 to 51880
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Comp.pagto.   11108 non-null  datetime64[ns]
 1   Result. pago  11108 non-null  float64       
 2   Unid. Orçam.  11108 non-null  int64         
 3   Proj/Ativ     11108 non-null  int64         
 4   Rubrica       11108 non-null  int64         
 5   Vinc. Orçam.  11108 non-null  int64         
 6   Elemento      11108 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(5)
memory usage: 694.2 KB


In [129]:
df_7001['Elemento'].unique()

array([339039, 319013, 449052, 339033, 339047, 339049, 319113, 339046,
       339036, 319011, 319016, 339035, 339014, 339092, 339030, 319092,
       339093, 339091, 319094, 339037, 332039, 339040, 449051, 319008,
       339147, 449040, 339048, 319007])

In [124]:
df_7001['Proj/Ativ'].unique()

array([2529, 2680, 9071, 2681, 4413, 4471, 4396, 2851, 2882, 4136])

In [130]:
df_7001['Vinc. Orçam.'].unique()

array([6069,    1])

In [131]:
df_7001.loc[df_7001['Vinc. Orçam.'] != 6069]

,Comp.pagto.,Result. pago,Unid. Orçam.,Proj/Ativ,Rubrica,Vinc. Orçam.,Elemento
45614,2022-01-25,11106462.01,7001,9071,339092470000,1,339047
45896,2022-02-25,860396.27,7001,9071,339047120100,1,339047
45964,2022-03-25,913204.14,7001,9071,339047120100,1,339047
46105,2022-04-22,920198.39,7001,9071,339047120100,1,339047
46238,2022-05-25,955590.41,7001,9071,339047120100,1,339047
46394,2022-06-24,895113.95,7001,9071,339047120100,1,339047
46643,2022-07-25,900446.96,7001,9071,339047120100,1,339047
46827,2022-08-25,1482772.89,7001,9071,339047120100,1,339047
46993,2022-09-23,923389.87,7001,9071,339047120100,1,339047
47146,2022-10-25,931011.43,7001,9071,339047120100,1,339047


In [34]:
df_rev = pd.read_csv(r'/content/drive/MyDrive/Dashboard_data/final_data/df_rev.csv', sep=';')
df_rev

,Data,vinculo,origem,tipo,nome_rubrica,valor_arrecadado
0,2004-01-01,400,Contribuições,Outras Contribuições Previdenciárias,Outras Contribuições Sociais,37.56
1,2004-01-01,6050,Contribuições Intra-Orçamentárias,Contr. Patronal - Servidor Civil Ativo - Princ...,Contr. Patronal - Servidor Ativo - Plano em Ca...,179881.44
2,2004-01-01,400,Outras Receitas Correntes,Outras Receitas Não Arrec.e Não Projetadas p/ ...,Outras Receitas - Diversas - Adm. Indireta,4699.39
3,2004-01-01,400,Contribuições Intra-Orçamentárias,Contr. Patronal - Servidor Civil Ativo - Princ...,Contr. Patronal - Servidor Ativo - Plano em Re...,2665961.14
4,2004-01-01,400,Contribuições,Contr.do Servidor Civil Ativo - Principal,Contr.do Servidor Civil Ativo - Plano em Repar...,2917709.38
...,...,...,...,...,...,...
13113,2025-03-01,6050,Contribuições Intra-Orçamentárias,Contr. Patronal - Servidor Civil Ativo - Princ...,Contr.Patronal Supl. - Serv. Ativo - Plano em ...,3.71
13114,2025-03-01,6049,Contribuições Intra-Orçamentárias,Contr. Patronal - Servidor Civil Ativo - Princ...,Contr. Patronal - Servidor Ativo - Plano em Re...,34921.77
13115,2025-03-01,6050,Contribuições,Contr.do Servidor Civil - Pensionistas - Princ...,Contr.do Servidor Civil - Pensionistas - Plano...,796.46
13116,2025-03-01,6049,Contribuições Intra-Orçamentárias,Contr. Patronal - Servidor Civil Ativo - Princ...,Contr. Patronal - Servidor Ativo - Plano em Re...,103926.64


In [28]:
# Generating a dataframe with paid expenses data from 2024 onwards

# Define the folder path
exp_folder_2024_onwards = pathlib.Path(r'/content/drive/MyDrive/Dashboard_data/Despesas/2024+')

# Read all CSV files efficiently
df_list = [
    pd.read_csv(expense_file).dropna(axis=1, how='all')
    for expense_file in exp_folder_2024_onwards.glob("*.csv")
]

# Concatenate all DataFrames
df_exp_2024_onwards = pd.concat(df_list, ignore_index=True)

# Convert financial columns efficiently
cols_to_convert = ['valorliquidado', 'restospagarnaoprocessadosliquidados']
df_exp_2024_onwards[cols_to_convert] = df_exp_2024_onwards[cols_to_convert].apply(
    lambda col: pd.to_numeric(col.str.replace('R\$ ', '', regex=True).str.replace(',', '.'), errors='coerce')
)

# Filter out rows with zero values in all columns to convert
df_exp_2024_onwards = df_exp_2024_onwards[
    df_exp_2024_onwards[cols_to_convert].ne(0).any(axis=1)
]

# Rename and clean up columns
df_exp_2024_onwards.rename(columns={
    'exercicio': 'YEAR', 'mes': 'MONTH'
}, inplace=True)

# Fix month values
df_exp_2024_onwards['MONTH'] = df_exp_2024_onwards['MONTH'].replace({0: 1, 13: 12}).astype(int)

# Convert date
df_exp_2024_onwards['Data'] = pd.to_datetime(df_exp_2024_onwards[['YEAR', 'MONTH']].assign(DAY=1))

# Sorting once before processing
df_exp_2024_onwards.sort_values('Data', inplace=True, ignore_index=True)

# Subtract the last nonzero appearance for target column
cols = ['subacao', 'naturezadespesa', 'fonterecursos', 'YEAR']
target_col = 'restospagarnaoprocessadosliquidados'
df_exp_2024_onwards[target_col + '_prev'] = (
    df_exp_2024_onwards.where(df_exp_2024_onwards[target_col] != 0).groupby(cols)[target_col].shift(1).fillna(0)
)
df_exp_2024_onwards[target_col] -= df_exp_2024_onwards[target_col + '_prev']


# df_exp_2024_onwards.loc[df_exp_2024_onwards['subacao'] == 4396, 'subacao'] = 2870
# df_exp_2024_onwards.loc[df_exp_2024_onwards['subacao'] == 4471, 'subacao'] = 2872
# df_exp_2024_onwards.loc[df_exp_2024_onwards['subacao'] == 4413, 'subacao'] = 2873

In [29]:
df_exp_2024_onwards = df_exp_2024_onwards[['Data', 'unidadeorcamentaria', 'subacao', 'naturezadespesa', 'valorliquidado', 'restospagarnaoprocessadosliquidados']]

In [10]:
len(df_exp_2024_onwards)

2213

In [16]:
df_exp_2024_onwards.columns

Index(['Data', 'unidadeorcamentaria', 'subacao', 'naturezadespesa',
       'valorliquidado', 'restospagarnaoprocessadosliquidados'],
      dtype='object')

In [8]:
df_exp_2024_onwards['naturezadespesa'].unique()

array([339037, 339039, 339047, 339040, 319192, 319007, 319013, 339030,
       339033, 339035, 339086, 319011, 319016, 319092, 319094, 339036,
       339046, 339048, 339049, 339092, 319113, 332039, 339014, 339093,
       449040, 449052, 319001, 319003, 319091, 999999, 339091, 339147])

In [30]:
df_exp_2024_onwards

,Data,unidadeorcamentaria,subacao,naturezadespesa,valorliquidado,restospagarnaoprocessadosliquidados
0,2024-01-01,7001,4396,319011,350329.90,0.0
1,2024-01-01,7002,2758,319003,8010.17,0.0
2,2024-01-01,7002,2758,319001,403383.48,0.0
3,2024-01-01,7002,2756,319092,8535.36,0.0
4,2024-01-01,7002,2756,319003,1258605.90,0.0
...,...,...,...,...,...,...
2208,2025-04-01,7001,2681,319113,111894.83,0.0
2209,2025-04-01,7001,2681,319113,34886.39,0.0
2210,2025-04-01,7003,9075,339047,0.00,0.0
2211,2025-04-01,7001,2529,339035,0.00,0.0


In [23]:
df_exp_2024_onwards

,Data,unidadeorcamentaria,subacao,naturezadespesa,valorliquidado,restospagarnaoprocessadosliquidados
0,2024-01-01,7001,4396,319011,350329.90,0.00
1,2024-01-01,7002,2758,319003,8010.17,0.00
2,2024-01-01,7002,2758,319001,403383.48,0.00
3,2024-01-01,7002,2756,319092,8535.36,0.00
4,2024-01-01,7002,2756,319003,1258605.90,0.00
...,...,...,...,...,...,...
2208,2025-04-01,7001,2681,319113,111894.83,0.00
2209,2025-04-01,7001,2681,319113,34886.39,0.00
2210,2025-04-01,7003,9075,339047,0.00,3599.35
2211,2025-04-01,7001,2529,339035,0.00,14300.00


In [11]:
arquivo = r'/content/drive/MyDrive/Previsoes_orcamento/despesas/ate_2023/Histórico de Despesas.xlsx'
df_7001 = pd.read_excel(arquivo,sheet_name='7001.t')
df_7002 = pd.read_excel(arquivo,sheet_name='7002.t')
df_7003 = pd.read_excel(arquivo,sheet_name='7003.t')

#adicionando Total nos df's originais, pois podemos fazer previsões só com ele
df_7001['Total'] = df_7001.iloc[:, 2:].sum(axis=1)
df_7002['Total'] = df_7002.iloc[:, 2:].sum(axis=1)
df_7003['Total'] = df_7003.iloc[:, 2:].sum(axis=1)

In [12]:
df_7001

,Data,T,7001-1373-339039,7001-1373-449052,7001-1503-339030,7001-1503-339035,7001-1503-339039,7001-1503-449051,7001-1503-449052,7001-1504-339036,...,7001-9042-332001,7001-9071-319091,7001-9071-339047,7001-9071-339091,7001-9071-339092,7001-9071-339093,7001-9085-319091,7001-9085-339091,7001-9090-319113,Total
0,2011-01-01,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,9808.44,28495.84,NaN,NaN,NaN,NaN,NaN,NaN,581973.30
1,2011-02-01,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2599.52,NaN,276953.25,NaN,NaN,NaN,NaN,NaN,NaN,914536.39
2,2011-03-01,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2599.52,3026.06,272912.56,NaN,NaN,NaN,NaN,NaN,NaN,981859.26
3,2011-04-01,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2599.52,NaN,321495.11,NaN,NaN,NaN,NaN,NaN,NaN,1055549.42
4,2011-05-01,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2599.52,NaN,276319.59,NaN,NaN,62.43,NaN,NaN,NaN,1015353.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,2023-08-01,152,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2132888.35,NaN,NaN,NaN,NaN,NaN,NaN,3648890.17
152,2023-09-01,153,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1540325.82,NaN,NaN,NaN,NaN,NaN,NaN,3219329.65
153,2023-10-01,154,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1534057.96,NaN,NaN,NaN,NaN,NaN,NaN,3138475.70
154,2023-11-01,155,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1551838.22,NaN,NaN,NaN,NaN,NaN,NaN,3085073.74
